# Northwind

Extenderemos nuestro ejemplo modelando nuestra base de datos en su formato dimensional o de estrella.

In [3]:
import sqlite3

# Connect to the DB
conn = sqlite3.connect("northwind.db")
c = conn.cursor();

In [4]:
c.execute("""SELECT name FROM sqlite_master WHERE type='table';""").fetchall()

[('Categories',),
 ('sqlite_sequence',),
 ('Customers',),
 ('Employees',),
 ('Shippers',),
 ('Suppliers',),
 ('Products',),
 ('Orders',),
 ('OrderDetails',)]

Crearemos las tablas de dimensiones con la información por la que querremos cortar o agrupar los datos. En este caso nuestras dimensiones serán las de **Cliente**, **Order** y **Producto**.

In [25]:
c.execute("""CREATE TABLE Customer_DIM AS SELECT CustomerID, CustomerName, Country, City FROM Customers;""").fetchall()
c.execute("""CREATE TABLE Order_DIM AS SELECT OrderID, datetime(OrderDate) as Date, ShipperName FROM Orders INNER JOIN Shippers""").fetchall()
c.execute("""CREATE TABLE Product_DIM AS SELECT ProductID, ProductName, Unit, Price FROM Products""").fetchall()

[]

Y los hechos a analizar, la información relativa a las **ordenes** (cuando, a qué precio, qué cantidad, etc...).

In [29]:
c.execute("""CREATE TABLE Orderline_FACT AS SELECT od.*, CustomerID FROM Orders o INNER JOIN OrderDetails od ON od.OrderID = o.OrderID""").fetchall()

[]

Vemos que ahora disponemos de nuevas tablas en nuestro sistema. Habitualmente estas tablas se encontrarán en otra base de datos, nuestro sistema informacional o _data warehouse_.

In [2]:
c.execute("""SELECT name FROM sqlite_master WHERE type='table';""").fetchall()

[('Categories',),
 ('sqlite_sequence',),
 ('Customers',),
 ('Employees',),
 ('Shippers',),
 ('Suppliers',),
 ('Products',),
 ('Orders',),
 ('OrderDetails',),
 ('Customer_DIM',),
 ('Order_DIM',),
 ('Product_DIM',),
 ('Orderline_FACT',)]

In [3]:
# !pip install sqlalchemy

In [4]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///northwind.db', echo=False)
connection = engine.connect()

Podemos observar la información relativa a las ordenes, en este caso solo disponemos de cantidad de producto vendida en cada orden.

In [31]:
import pandas as pd

fact_table = pd.read_sql_table("Orderline_FACT", con=engine)
fact_table

,OrderDetailID,OrderID,ProductID,Quantity,CustomerID
0,1,10248,11,12,90
1,2,10248,42,10,90
2,3,10248,72,5,90
3,4,10249,14,9,81
4,5,10249,51,40,81
...,...,...,...,...,...
513,514,10442,11,30,20
514,515,10442,54,80,20
515,516,10442,66,60,20
516,517,10443,11,6,66


Pero combinando con la información de contexto podemos obtener información significativa a las preguntas que nos haciamos antes.

In [5]:
import pandas as pd

query = """
SELECT 	strftime('%m', Date) AS Month, CustomerID AS Client, ProductID AS Product, count(*) AS Howmany 
FROM Orderline_FACT
JOIN Order_DIM 
GROUP BY Month, Client, Product
"""
pd.read_sql(query, con=connection)

,Month,Client,Product,Howmany
0,01,2,69,99
1,01,2,70,99
2,01,3,11,99
3,01,4,13,99
4,01,4,24,99
...,...,...,...,...
3867,12,90,11,93
3868,12,90,42,93
3869,12,90,72,93
3870,12,91,31,93


In [6]:
query = """
SELECT 	strftime('%m', Date) AS Month, CustomerID AS Client, ProductName, count(*) AS Howmany 
FROM Orderline_FACT
JOIN Order_DIM 
JOIN Product_DIM p
GROUP BY Month, Client, ProductName
"""
pd.read_sql(query, con=connection)

,Month,Client,ProductName,Howmany
0,01,2,Alice Mutton,198
1,01,2,Aniseed Syrup,198
2,01,2,Boston Crab Meat,198
3,01,2,Camembert Pierrot,198
4,01,2,Carnarvon Tigers,198
...,...,...,...,...
45579,12,91,Uncle Bob's Organic Dried Pears,186
45580,12,91,Valkoinen suklaa,186
45581,12,91,Vegie-spread,186
45582,12,91,Wimmers gute Semmelknödel,186


In [7]:
query = """
SELECT strftime('%Y', Date) AS Year, 
       strftime('%m', Date) AS Month,
       CustomerName AS Client, 
       ProductName AS Product, 
       count(*) AS Howmany
FROM Orderline_FACT fact
JOIN Order_DIM o ON o.OrderID = fact.OrderID
JOIN Product_DIM p ON p.ProductID = fact.ProductID
JOIN Customer_DIM c ON c.CustomerID = fact.CustomerID
GROUP BY Year, Month, Client, Product
"""
pd.read_sql(query, con=connection)

,Year,Month,Client,Product,Howmany
0,1996,07,Blondel père et fils,Alice Mutton,3
1,1996,07,Blondel père et fils,Outback Lager,3
2,1996,07,Centro comercial Moctezuma,Gravad lax,3
3,1996,07,Centro comercial Moctezuma,Sir Rodney's Scones,3
4,1996,07,Chop-suey Chinese,Guaraná Fantástica,3
...,...,...,...,...,...
507,1997,02,Save-a-lot Markets,Thüringer Rostbratwurst,3
508,1997,02,Toms Spezialitäten,Ravioli Angelo,3
509,1997,02,Toms Spezialitäten,Sasquatch Ale,3
510,1997,02,Toms Spezialitäten,Teatime Chocolate Biscuits,3


Estas consultas sin disponer de mucho dato pueden ser pesadas. En estos casos suele ser recomendable, si no nos urge la información de rabiosa actualidad, que estos procesos sean ejecutados durante la noche para disponer de información ya agregada a la mañana siguiente.

# Cubos

Se definen como cubos agregaciones como la anterior ya consolidadas como tablas donde podemos cortar las dimensiones en base a los datos de corte dimensional seleccionados.

In [10]:
query = """
SELECT strftime('%Y', Date) AS Year, 
       strftime('%m', Date) AS Month,
       CustomerName AS Client, 
       ProductName AS Product, 
       count(*) AS Howmany
FROM Orderline_FACT fact
JOIN Order_DIM o ON o.OrderID = fact.OrderID
JOIN Product_DIM p ON p.ProductID = fact.ProductID
JOIN Customer_DIM c ON c.CustomerID = fact.CustomerID
GROUP BY Year, Month, Client, Product
"""
productivity = pd.read_sql(query, con=connection)
productivity.to_sql("Productivity_CUBE", con=engine)

512

In [13]:
# Now it should be faster to get our insights
# Two most productive months
query = """SELECT Month, SUM(Howmany) AS Cuantos
FROM Productivity_CUBE 
GROUP BY Month
ORDER BY 2 DESC
LIMIT 2"""
pd.read_sql(query, con=connection)

,Month,Cuantos
0,01,255
1,12,243


In [14]:
# Three most productive months-client
query = """SELECT Month, Client, SUM(Howmany) AS Cuantos
FROM Productivity_CUBE 
GROUP BY Month, Client
ORDER BY 3 DESC
LIMIT 3"""
pd.read_sql(query, con=connection)

,Month,Client,Cuantos
0,08,QUICK-Stop,30
1,09,Hungry Owl All-Night Grocers,27
2,10,Frankenversand,27


In [15]:
conn.close()